
## The Best Area to Live in LA

1. Introduction
Moving to a new city is always exciting and exhausting. It means the beginning of a new life. It also means that you need to find a neighborhood to live in and start your new life. I was moving to Los Angeles from an out-of-state a year ago, and the scene of searching for a new neighborhood online was still vivid. Due to my unfamiliarity with the city, I spent a lot of time searching the Internet for the most suitable area to live in Los Angeles. In this project, I intend to collect data from various sources, analyze and cluster neighborhoods of Los Angeles based on safety, convenience and economy in order to provide some useful information to people who have just come to this city or want to find a new place to live.

1.1 Business Problem
In order to make this project more applicative, here we will concretize the problem we are about to solve, and introduce some assumptions and constraints.
For choosing the most suitable area/neighborhood to live, the factors we consider include:

Community safety: The lower the crime rate per capita, the more suitable the area to live.
Convenience level: We will measure the number of grocery stores, restaurants and shopping malls in the neighborhood.
Entertainment activities: We will measure the number of art and entertainment venues, outdoor recreation venues and nightlife spots in the community.
Rent level: we will tend to live in areas where the average rent is relatively low.
1.2 Target Audience
This project is aimed at singles or families without children who want to relocate to Los Angeles. So educational resources are not considered for the time being. In addition, due to the pandemic, most people are able to work from home. So the impact of commuting time on location selection is not considered here. Last but not least, the audience of this project is not limited to newcomers who have just arrived in LA. It is also instructive for people who already live in LA but want to reconsider their residential communities.

2. Source of Data
In this project, we will fetch or extract data from the following sources:
    List of regions and neighborhoods in Los Angeles: https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_in_Los_Angeles
Los Angeles Rental Market Trends: https://www.rentcafe.com/average-rent-market-trends/us/ca/los-angeles/
Violent crime in the City of Los Angeles from December 30, 2019 to June 28, 2020: https://maps.latimes.com/neighborhoods/violent-crime/neighborhood/list/
Number of Arts & Entertainment, Food, Nightlife Spot, Outdoors & Recreation, and Shop & Service in every neighborhood - Foursquare API
Coordinates of all neighborhoods and venues - GeoPy Nominatim geocoding
3. Methodology
In essence, our methodology is to cluster the neighborhoods in Los Angeles based on the crime rate, rent, and the number of different types of venues in each neighborhood to obtain a couple of clusters. In the end, readers can choose a neighborhood from a certain cluster that suits them best according to the factors they value.

4. Analysis and Modeling
4.1 Import Required Libraries

## Import Required Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup
import re

!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                                                /failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|1_73_pypy|2_73_pypy|3_73_pypy|4_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']
  - rsa -

### 4.2 Neighborhoods Data

#### 4.2.1 Scrape and clean up the list of neighborhoods


First, we need to scrap a list of neighborhoods from the Wikipedia page-List of districts and neighborhoods in Los Angeles

In [2]:
# download data and parse it:
url = requests.get('https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_in_Los_Angeles').text
soup = BeautifulSoup(url, "html.parser")

lis = []
for li in soup.findAll('li'):
    if li.find(href="/wiki/Portal:Los_Angeles"):
        break
    if li.find(href=re.compile("^/wiki/")):
        lis.append(li)
    if li.text=='Pico Robertson[34]': #Pico Robertson is the only item on the list that does not have a hyperlink reference
        lis.append(li)
len(lis)

200

In [3]:
neigh = []
for i in range(0,len(lis)):
    neigh.append(lis[i].text.strip())
    
df = pd.DataFrame(neigh)
df.columns = ['Neighborhood']
df.head()

,Neighborhood
0,Angelino Heights[1]
1,Angeles Mesa[2]
2,Angelus Vista[2]
3,Arleta[3][1]
4,Arlington Heights[3]


In [4]:
df['Neighborhood'] = df.Neighborhood.str.partition('[')[0] #Removes the citation and reference brackets
df['Neighborhood'] = df.Neighborhood.str.partition(',')[0] #Removes the alternatives for 'Bel Air'
df=df[df.Neighborhood!='Baldwin Hills/Crenshaw'] #Removes redundancy as 'Baldwin Hills' and 'Crenshaw' exist already
df=df[df.Neighborhood!='Hollywood Hills West'] #Removes redundancy as it has the same coordinates as 'Hollywood Hills'
df=df[df.Neighborhood!='Brentwood Circle'] #Removes redundancy as it has the same coordinates as 'Brentwood'
df=df[df.Neighborhood!='Wilshire Park'] #Removes redundancy as it has the same coordinates as 'Wilshire Center'
df.reset_index(inplace=True,drop=True)
df.head()

,Neighborhood
0,Angelino Heights
1,Angeles Mesa
2,Angelus Vista
3,Arleta
4,Arlington Heights


Get the latitude and the longitude coordinates of each neighborhood.

In [5]:
# define the data frame columns
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the data frame
nhoods = pd.DataFrame(columns=column_names)

# use GeoPy Nominatim geolocator with the user_agent "la_explorer".
geolocator = Nominatim(user_agent="la_explorer",timeout=5)
for i in range(0,len(df)):
    
    address = df.Neighborhood[i]+', Los Angeles'
    location = geolocator.geocode(address)
    if location == None:
        latitude = 0
        longitude = 0
    else:
        latitude = location.latitude
        longitude = location.longitude

    nhoods = nhoods.append({'Neighborhood': df.Neighborhood[i],
                                              'Latitude': latitude,
                                              'Longitude': longitude}, ignore_index=True)
print("The number of neighborhood before clean up is:",  len(nhoods))
nhoods.head()

The number of neighborhood before clean up is: 196


,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,Angeles Mesa,33.991402,-118.319520
2,Angelus Vista,-23.403598,-51.965818
3,Arleta,34.241327,-118.432205
4,Arlington Heights,34.043494,-118.321374


In [6]:
# clean neighbourhood data by deleting missing values and obvious geocoding errors
nhoods['Latitude']=nhoods['Latitude'].astype(float)
nhoods['Longitude']=nhoods['Longitude'].astype(float)

nhoods=nhoods[(nhoods.Latitude>33.5) & (nhoods.Latitude<34.4) & (nhoods.Longitude<-118)] 
nhoods.reset_index(inplace=True,drop=True)
nhoods.head()
print("The number of neighborhood after clean up is:",  len(nhoods))
nhoods

The number of neighborhood after clean up is: 161


,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,Angeles Mesa,33.991402,-118.319520
2,Arleta,34.241327,-118.432205
3,Arlington Heights,34.043494,-118.321374
4,Arts District,34.041239,-118.234450
5,Atwater Village,34.116398,-118.256464
6,Baldwin Hills,34.017616,-118.381694
7,Baldwin Village,34.019431,-118.345803
8,Beachwood Canyon,34.122292,-118.321384
9,Bel Air,34.082728,-118.447980


### 4.2.2 Plot LA neighborhood map

#### Get the coordinates of the center of Los Angeles

In [7]:
#Use geopy library to get the latitude and longitude values of Los Angeles.
address = 'Los Angeles, USA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.242766.


#### Create a map of Los Angeles with the neighborhood superimposed on top.

In [8]:
# create map of LA using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(nhoods['Latitude'], nhoods['Longitude'], nhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

### 4.3 Crime Data

To analyze the safety of the community, we will use the violent crime rate per 10,000 people from December 30, 2019 to June 28, 2020. The table below contains both per capita statistics and gross crime counts.

In [16]:
crime_df=

SyntaxError: invalid syntax (<ipython-input-16-5bb2a53ca13a>, line 1)

In [10]:
# merge neighbohood data with crime data
nhoods_crime=pd.merge(nhoods, crime_df, on='Neighborhood')

nhoods_crime.head()

NameError: name 'crime_df' is not defined

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns

crime_top10=nhoods_crime.sort_values(['CrimePerCapita'], ascending=True).head(10)

plt.figure(figsize=(10, 6))

sns.set(style="white", context="talk", palette="rocket")

sns.barplot(
    data=crime_top10,
    x=crime_top10["Neighborhood"],
    y=crime_top10["CrimePerCapita"],
)

plt.xticks(rotation=45, ha='right')
sns.despine(offset=10, trim=True, bottom=True)
plt.tight_layout(h_pad=2)

NameError: name 'nhoods_crime' is not defined

In [11]:
body = client_936d2e8b22754b78ae27e343371ab102.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-hnyptdidd5gx3a',Key='average_rent_in_la.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

rent_df = pd.read_csv(body)
rent_df.head()

NameError: name 'client_936d2e8b22754b78ae27e343371ab102' is not defined

In [18]:
# merge rent data with neighbohood and crime data
nhoods_crime_rent=pd.merge(nhoods_crime, rent_df, on='Neighborhood')

nhoods_crime_rent.head()

NameError: name 'nhoods_crime' is not defined

In [19]:
rent_top10=nhoods_crime_rent.sort_values(['Average Rent'], ascending=True).head(10)

plt.figure(figsize=(10, 6))

sns.set(style="white", context="talk", palette="rocket")

sns.barplot(
    data=crime_top10,
    x=rent_top10["Neighborhood"],
    y=rent_top10["Average Rent"],
)

plt.xticks(rotation=45, ha='right')
sns.despine(offset=10, trim=True, bottom=True)
plt.tight_layout(h_pad=2)

NameError: name 'nhoods_crime_rent' is not defined

### 4.5 Venues Data (Foursquare API)

In [20]:
CLIENT_ID = '' #  Foursquare ID
CLIENT_SECRET = '' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [13]:
#create a function to explore all neighborhoods in LA.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
#write the code to run the above function on each neighborhood and create a new dataframe called toronto_denc_venues
la_venues = getNearbyVenues(names=nhoods_crime_rent['Neighborhood'],
                                   latitudes=nhoods_crime_rent['Latitude'],
                                   longitudes=nhoods_crime_rent['Longitude']
                                  )
la_venues.head()

NameError: name 'nhoods_crime_rent' is not defined

In [22]:
la_venues.drop_duplicates(keep="first", inplace=True)
len(la_venues)

NameError: name 'la_venues' is not defined

In [23]:
#find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(la_venues['Venue Category'].unique())))

NameError: name 'la_venues' is not defined

In [24]:
def get_categories():
    try:
        with open("categories.json") as data:
            categories = json.load(data)
    except IOError:
        url = 'https://api.foursquare.com/v2/venues/categories'
        params = {
            "client_id": CLIENT_ID,
            "client_secret": CLIENT_SECRET,
            "v": VERSION,
        }
        categories = requests.get(url, params=params).json()["response"]["categories"]
    return categories

In [25]:
#the function return dictionaries of lists with parents and child categories
def collect_categories(node, categories):
    categories.append(node["name"])
    if not node["categories"]:
        return
    for sub_node in node['categories']:
        collect_categories(sub_node, categories)

In [26]:
#from list of dictionaries to one dictoinary
categories_list = {}
for i in get_categories():
    categories = []
    collect_categories(i, categories)
    categories_list[i["name"]] = categories

KeyError: 'categories'

In [27]:
venueCat = []

for venue_category in la_venues["Venue Category"]:
    for key in categories_list.keys():
        if venue_category in categories_list[key]:
            venueCat.append(key)

la_venues["General Venue Category"] = venueCat

la_venues.head(10)

NameError: name 'la_venues' is not defined

In [28]:
#find out what th genetal venue categories are
print('General Venue Categories include {}'.format((la_venues['General Venue Category'].unique())))

NameError: name 'la_venues' is not defined

In [29]:
la_venues.drop(
    la_venues[
        (la_venues["General Venue Category"] == 'Travel & Transport') |
        (la_venues["General Venue Category"] == 'Professional & Other Places') |
        (la_venues["General Venue Category"] == 'Residence') |
        (la_venues["General Venue Category"] == 'College & University')
    ].index,
    axis=0,
    inplace=True,
)
 
la_venues.head(10)

NameError: name 'la_venues' is not defined

In [30]:
colorDict = {
    'Arts & Entertainment': 'yellow',
    'Food': 'orange',
    'Shop & Service': 'red',
    'Outdoors & Recreation': 'cadetblue',
    'Nightlife Spot': 'purple'
}

In [31]:
from folium.plugins import MarkerCluster

venueMap = folium.Map(
    location=[latitude, longitude], 
    zoom_start=10
)

markCluster = MarkerCluster().add_to(venueMap)

for lat, lng, cat in zip(la_venues['Venue Latitude'],
                         la_venues['Venue Longitude'],
                         la_venues['General Venue Category']):  
    if cat in colorDict:
        folium.Marker(
            location=[lat, lng],
            icon=folium.Icon(color=colorDict[cat]),
        ).add_to(markCluster)
        
venueMap

NameError: name 'la_venues' is not defined

##### Analyze each neighborhood

In [32]:
# one hot encoding
la_onehot = pd.get_dummies(la_venues[['General Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot['Neighborhood'] = la_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1]) 
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

NameError: name 'la_venues' is not defined

In [33]:
#group rows by neighborhood and by taking the sum of occurrence of each category
la_grouped = la_onehot.groupby('Neighborhood').sum().reset_index()

la_grouped.head()
len(la_grouped)

NameError: name 'la_onehot' is not defined

In [34]:
arts_top10=la_grouped.sort_values(['Arts & Entertainment'], ascending=False).head(10)

plt.figure(figsize=(10, 7))

sns.set(style="white", context="talk", palette="rocket")

sns.barplot(
    data=arts_top10,
    x=arts_top10["Neighborhood"],
    y=arts_top10["Arts & Entertainment"],
)

plt.xticks(rotation=45, ha='right')
sns.despine(offset=10, trim=True, bottom=True)
plt.tight_layout(h_pad=2)

NameError: name 'la_grouped' is not defined

In [35]:
food_top10=la_grouped.sort_values(['Food'], ascending=False).head(10)

plt.figure(figsize=(10, 7))

sns.set(style="white", context="talk", palette="rocket")

sns.barplot(
    data=food_top10,
    x=food_top10["Neighborhood"],
    y=food_top10["Food"],
)

plt.xticks(rotation=45, ha='right')
sns.despine(offset=10, trim=True, bottom=True)
plt.tight_layout(h_pad=2)

NameError: name 'la_grouped' is not defined

In [36]:
night_top10=la_grouped.sort_values(['Nightlife Spot'], ascending=False).head(10)

plt.figure(figsize=(10, 7))

sns.set(style="white", context="talk", palette="rocket")

sns.barplot(
    data=night_top10,
    x=night_top10["Neighborhood"],
    y=night_top10["Nightlife Spot"],
)

plt.xticks(rotation=45, ha='right')
sns.despine(offset=10, trim=True, bottom=True)
plt.tight_layout(h_pad=2)

NameError: name 'la_grouped' is not defined

In [37]:
outdoors_top10=la_grouped.sort_values(['Outdoors & Recreation'], ascending=False).head(10)

plt.figure(figsize=(10, 7))

sns.set(style="white", context="talk", palette="rocket")

sns.barplot(
    data=outdoors_top10,
    x=outdoors_top10["Neighborhood"],
    y=outdoors_top10["Outdoors & Recreation"],
)

plt.xticks(rotation=45, ha='right')
sns.despine(offset=10, trim=True, bottom=True)
plt.tight_layout(h_pad=2)


NameError: name 'la_grouped' is not defined

In [38]:
shop_top10=la_grouped.sort_values(['Shop & Service'], ascending=False).head(10)

plt.figure(figsize=(10, 7))

sns.set(style="white", context="talk", palette="rocket")

sns.barplot(
    data=shop_top10,
    x=shop_top10["Neighborhood"],
    y=shop_top10["Shop & Service"],
)

plt.xticks(rotation=45, ha='right')
sns.despine(offset=10, trim=True, bottom=True)
plt.tight_layout(h_pad=2)

NameError: name 'la_grouped' is not defined

#### 4.6 Cluster Analysis

In [39]:
la_df=pd.merge(nhoods_crime_rent, la_grouped, on='Neighborhood')
la_df.drop_duplicates(keep="first", inplace=True)
la_df.head()

NameError: name 'nhoods_crime_rent' is not defined

In [40]:
#create the new dataframe and display the top 1 venue for each neighborhood.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 1

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
la_sorted = pd.DataFrame(columns=columns)
la_sorted['Neighborhood'] = la_grouped['Neighborhood']

for ind in np.arange(la_grouped.shape[0]):
    la_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped.iloc[ind, :], num_top_venues)

la_sorted.drop_duplicates(keep="first", inplace=True)
la_sorted.head()

NameError: name 'la_grouped' is not defined

Normalizing over the standard deviation

In [41]:
la_cluster = la_df.drop(columns=["Neighborhood", "Latitude", "Longitude","CrimeCounts"])

from sklearn.preprocessing import StandardScaler
X = la_cluster.values[:,1:]
X = np.nan_to_num(X)
Clus_dataSet = StandardScaler().fit_transform(X)

NameError: name 'la_df' is not defined

### Modeling
One important point for cluster analysis is to determine the optimal value of K. We will use Silhouette Score and Sum of Squared Distance to help us decide.

Sum of Squared Distance measures error between data points and their assigned clusters’ centroids. The smaller the better.

Silhouette Score focuses on minimizing the sum of squared distance inside the cluster as well, meanwhile, it also tries to maximize the distance between its neighborhoods. A higher Silhouette Score indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.

From the figure below, we can see that when K=2, the Silhouette Score is the highest, but the Sum of Squared Distance is also very high. We would choose K=5, since this number balances the Silhouette Score and the Sum of Squared Distance.

In [42]:
# Evaluate KMeans first to choose K
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score


sse = []
sil = []
list_k = list(range(2, 12))

for k in list_k:
    km = KMeans(n_clusters=k, random_state=0)
    km.fit(Clus_dataSet)
    sse.append(km.inertia_)
    sil.append(silhouette_score(Clus_dataSet, km.labels_))

fig, ax1 = plt.subplots(figsize=(6, 6))

ax2 = ax1.twinx()
ax1.plot(list_k, sse, 'bo-')
ax2.plot(list_k, sil, 'rd-')

ax1.set_xlabel(r'Number of clusters *k*')
ax1.set_ylabel('Sum of inner squared distance', color='b')
ax2.set_ylabel('Silhouette score', color='r')

plt.show()


NameError: name 'Clus_dataSet' is not defined

In [43]:
#Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Clus_dataSet)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

NameError: name 'Clus_dataSet' is not defined

In [44]:
#create a new dataframe that includes the cluster as well as the top 1 venues for each neighborhood.

# add clustering labels
la_df.insert(0, 'Cluster Labels', kmeans.labels_)

# merge la_df with la_sorted to add top 1 venues for each neighborhood
la_merged=pd.merge(la_df, la_sorted, on='Neighborhood')
la_merged.head()

NameError: name 'la_df' is not defined

In [45]:
#visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        la_merged['Latitude'], 
        la_merged['Longitude'], 
        la_merged['Neighborhood'], 
        la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'la_merged' is not defined

### Examine Clusters

In [46]:
la_cluster_mean = la_merged.drop(columns=["Neighborhood", "Latitude", "Longitude","CrimeCounts", "1st Most Common Venue"])
la_cluster_mean.groupby('Cluster Labels').mean()

NameError: name 'la_merged' is not defined

## 5. Results

### Cluster 1

In [47]:
la_merged.loc[la_merged['Cluster Labels'] == 0, la_merged.columns[[1] + [4] +list(range(6, la_merged.shape[1]))]]

NameError: name 'la_merged' is not defined

### Cluster 2

In [48]:
la_merged.loc[la_merged['Cluster Labels'] == 1, la_merged.columns[[1] + [4] +list(range(6, la_merged.shape[1]))]]

NameError: name 'la_merged' is not defined

###  Cluster 3

In [49]:
la_merged.loc[la_merged['Cluster Labels'] == 2, la_merged.columns[[1] + [4] +list(range(6, la_merged.shape[1]))]]

NameError: name 'la_merged' is not defined

### Cluster 4

In [50]:
la_merged.loc[la_merged['Cluster Labels'] == 3, la_merged.columns[[1] + [4] +list(range(6, la_merged.shape[1]))]]

NameError: name 'la_merged' is not defined

### Cluster 5

In [51]:
la_merged.loc[la_merged['Cluster Labels'] == 4, la_merged.columns[[1] + [4] +list(range(6, la_merged.shape[1]))]]

NameError: name 'la_merged' is not defined

### Combining the previous centroid values, we can find that each cluster has its own characteristics.

Cluster 1- has the lowest crime rate and medium rent. This cluster has the greatest number of shop & service among 5 clusters. In addition, number of food venues are significantly higher than other venue types.
Cluster 2- has a moderate crime rate and the lowest average rent. However, the number of various venues is the least among 5 clusters, which is not convenient

Cluster 3- has the highest crime rate and the highest average rent. But it also has the most art & entertainment and outdoors & recreation venues. Obviously, it is a good place for art immersion and outdoor relaxation.

Cluster 4- has the second highest crime rate and average rent. The number of food and nightlife spot venues are the most among 5 clusters. It is a good choice for foodies and nightlife lovers.

Cluster 5- has a moderately low crime rate and the second highest rent. The number of food and shop & service in the cluster is significantly more than other venues.

## 6. Discussion

Based on the above analysis, in my opinion cluster 1 is a cost-effective choice. Not only because I am most concerned about safety when I personally choose the living place, but also because it has second lowest rent among all clusters. Compared to the lowest rent cluster 2, it has more food and shops, which means living in cluster 1 is more convenient.

Of course, if someone simply wants to pursue the lowest rent and doesn't care about venues in the neighborhood, I would recommend them to choose from cluster 2.

Cluster 3 seems to me a paradise for artists and outdoor enthusiasts, even if it has no advantage in crime rate and rent. There is only one neighborhood - Exposition Park in this cluster. A simple search reveal that this area has Los Angeles Memorial Coliseum, Los Angeles Memorial Coliseum, California Science Center, Lucas Museum of Narrative Art, Exposition Park Rose Garden and so on. It is not difficult to understand why it is so special.

If you are a gourmet or like to hang out at night, I would definitely recommend you choose from cluster 4. This category includes Downtown and Hollywood. Living here can definitely satisfy your appetite for delicious food and yearning for bars.

Cluster 5 seems to be inferior to cluster 1 in all aspects (for example, the rent is not as favorable as cluster 1, but the crime rate is higher than cluster 1), however, it has more outdoor & recreation venues than cluster 1. From the map, we can see that the neighborhoods in cluster 5 are mostly located in areas closer to the beach. If you like to enjoy the romance of the beach, I would recommend you to choose from cluster 5.

## 7. Conclusion
The objective of this project is to find the most livable area in Los Angeles. By acquiring data from different sources, processing and cleaning them into a data frame containing per capita crime rate, average rent, and number of various venues, we were able to apply K-Mean clustering algorithm and finally get 5 clusters.

We analyzed the characteristics of 5 clusters, and on this basis, recommended a suitable neighborhood cluster for target audiences with different needs. I hope that this analysis will be beneficial for people who have just arrived in Los Angeles or who are considering moving to a new place.